In [1]:
import pandas as pd
%pip install requests
import requests
# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head(100)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


/Users/kanbesouichirou/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-11T13:01:35.358Z,63.099800,-150.319700,89.20,2.00,ml,NaN,NaN,NaN,0.67,...,2025-11-11T13:12:07.076Z,"70 km NNE of Petersville, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
1,2025-11-11T12:31:07.010Z,36.582668,-121.045166,7.69,1.53,md,20.0,99.0,0.006145,0.23,...,2025-11-11T12:57:18.742Z,"10 km ENE of Pinnacles, CA",earthquake,0.54,1.09,0.150000,16.0,automatic,nc,nc
2,2025-11-11T12:26:00.720Z,33.486500,-116.811333,5.70,0.89,ml,56.0,19.0,0.016910,0.19,...,2025-11-11T12:29:30.181Z,"7 km NE of Aguanga, CA",earthquake,0.14,0.42,0.160000,26.0,automatic,ci,ci
3,2025-11-11T12:13:51.600Z,32.922833,-116.235333,8.22,0.73,ml,33.0,46.0,0.065410,0.19,...,2025-11-11T12:17:11.814Z,"26 km SSW of Ocotillo Wells, CA",earthquake,0.24,0.60,0.221000,9.0,automatic,ci,ci
4,2025-11-11T12:12:13.120Z,37.597000,-122.468170,10.13,0.97,md,7.0,111.0,0.028990,0.02,...,2025-11-11T12:32:19.589Z,"2 km SE of Pacifica, CA",earthquake,0.57,1.51,0.290000,6.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-11-10T23:26:02.590Z,38.821499,-122.809830,2.69,0.76,md,12.0,66.0,0.017640,0.05,...,2025-11-11T00:07:19.520Z,"7 km NW of The Geysers, CA",earthquake,0.37,0.86,0.190000,13.0,automatic,nc,nc
96,2025-11-10T23:25:44.824Z,62.198400,-148.435700,31.20,1.50,ml,NaN,NaN,NaN,0.61,...,2025-11-10T23:28:07.397Z,"44 km N of Chickaloon, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
97,2025-11-10T23:23:12.620Z,46.805833,-121.740333,0.56,0.57,ml,16.0,133.0,0.019440,0.10,...,2025-11-11T04:16:14.030Z,"22 km NNW of Packwood, Washington",earthquake,0.29,0.54,0.220835,6.0,reviewed,uw,uw
98,2025-11-10T23:05:08.410Z,19.188500,-155.592167,8.41,2.25,ml,31.0,95.0,0.061230,0.14,...,2025-11-10T23:22:22.660Z,"12 km W of Pāhala, Hawaii",earthquake,0.29,0.81,0.144311,18.0,reviewed,hv,hv
